<a href="https://colab.research.google.com/github/Akash-mahandargi/Basic-Projects-/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Example usage of the get_recommendations function
example_anime_title = 'Fullmetal Alchemist: Brotherhood'  # Replace with an anime title from your dataset
recommendations = get_recommendations(example_anime_title, cosine_sim, df_anime)

print(f"Recommendations for '{example_anime_title}':")
if recommendations:
    for i, rec in enumerate(recommendations):
        print(f"{i+1}. {rec}")
else:
    print("No recommendations found.")

Recommendations for 'Fullmetal Alchemist: Brotherhood':
1. Fullmetal Alchemist
2. Shingeki no Kyojin
3. Fairy Tail
4. Angel Beats!
5. Naruto
6. Code Geass: Hangyaku no Lelouch
7. Ao no Exorcist
8. Soul Eater
9. Bleach
10. Sword Art Online


In [1]:
import pandas as pd

df_anime = pd.read_csv('anime.csv')
display(df_anime.head())

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [2]:
print(df_anime.isnull().sum())

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64


In [3]:
df_anime.dropna(subset=['genre', 'type', 'rating'], inplace=True)
print(df_anime.isnull().sum())

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


In [4]:
display(df_anime.info())
display(df_anime.describe())
display(df_anime['type'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


None

,anime_id,rating,members
count,12017.000000,12017.000000,1.201700e+04
mean,13638.001165,6.478264,1.834888e+04
std,11231.076675,1.023857,5.537250e+04
min,1.000000,1.670000,1.200000e+01
25%,3391.000000,5.890000,2.250000e+02
50%,9959.000000,6.570000,1.552000e+03
75%,23729.000000,7.180000,9.588000e+03
max,34519.000000,10.000000,1.013917e+06


,count
type,
TV,3668
OVA,3284
Movie,2259
Special,1670
ONA,648
Music,488


In [5]:
# Step 1 & 2: Select and preprocess the 'genre' column using one-hot encoding
# Handle multiple genres per anime by splitting the genre string and creating a set of unique genres
all_genres = set()
for genres in df_anime['genre'].str.split(', '):
    if genres is not None:
        all_genres.update(genres)

# Create a binary matrix for genres
for genre in all_genres:
    df_anime[genre] = df_anime['genre'].apply(lambda x: 1 if genre in x else 0)

# Step 3: Drop the original 'genre' column
df_anime = df_anime.drop('genre', axis=1)

# Step 4 & 5: Select other relevant numerical features and combine with one-hot encoded genres
# Ensure 'rating' and 'members' are numerical types before selecting
df_anime['rating'] = pd.to_numeric(df_anime['rating'], errors='coerce')
df_anime['members'] = pd.to_numeric(df_anime['members'], errors='coerce')

# Drop rows where conversion to numeric resulted in NaN for 'rating' or 'members'
df_anime.dropna(subset=['rating', 'members'], inplace=True)

# Select all genre columns, 'rating', and 'members' for similarity computation
genre_columns = list(all_genres)
features_for_similarity = df_anime[genre_columns + ['rating', 'members']]

display(features_for_similarity.head())

,Shoujo,Demons,Hentai,Military,Yuri,Samurai,Ecchi,Sports,Sci-Fi,Music,...,Game,Yaoi,Shounen Ai,School,Slice of Life,Adventure,Josei,Drama,rating,members
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,9.37,200630
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,9.26,793665
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,9.25,114262
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,9.17,673572
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,9.16,151266


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(features_for_similarity)
print(cosine_sim.shape)

(12017, 12017)


In [7]:
def get_recommendations(title, cosine_sim=cosine_sim, df=df_anime):
    # Get the index of the anime that matches the title
    if title not in df['name'].values:
        print(f"Anime with title '{title}' not found in the dataset.")
        return []

    idx = df[df['name'] == title].index[0]

    # Get the pairwise similarity scores for all anime with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar anime (excluding the input anime)
    # Take the top 11 to exclude the anime itself
    sim_scores = sim_scores[1:11]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the titles of the top 10 most similar anime
    return df['name'].iloc[anime_indices].tolist()

# Example usage (optional - uncomment to test)
# recommendations = get_recommendations('Fullmetal Alchemist: Brotherhood')
# print(recommendations)

In [8]:
# Step 2: Define functions to calculate precision, recall, and F1-score
def calculate_precision(recommended_anime, relevant_anime):
    """Calculates precision."""
    if not recommended_anime:
        return 0.0
    true_positives = len(set(recommended_anime) & set(relevant_anime))
    return true_positives / len(recommended_anime)

def calculate_recall(recommended_anime, relevant_anime):
    """Calculates recall."""
    if not relevant_anime:
        return 0.0
    true_positives = len(set(recommended_anime) & set(relevant_anime))
    return true_positives / len(relevant_anime)

def calculate_f1_score(precision, recall):
    """Calculates F1-score."""
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

# Step 1, 3 & 4: Simulate user interactions and evaluate
# Define a test set (a few example anime titles to get recommendations for)
test_anime_titles = [
    'Kimi no Na wa.',
    'Fullmetal Alchemist: Brotherhood',
    'Steins;Gate',
    'Spirited Away',
    'Attack on Titan'
]

# Define what constitutes "relevant" anime for evaluation.
# For this content-based approach, we'll consider anime with a high similarity score (e.g., > 0.7)
# to the target anime as relevant. This is a simplification for evaluation purposes without user data.
# We'll calculate relevant anime for each test case dynamically.

all_precisions = []
all_recalls = []
all_f1_scores = []

# Iterate through the test set
for anime_title in test_anime_titles:
    print(f"Evaluating recommendations for: {anime_title}")

    # Get recommendations using the previously defined function
    recommended_anime = get_recommendations(anime_title, cosine_sim, df_anime)
    #print(f"Recommended: {recommended_anime}") # Uncomment to see recommendations

    if not recommended_anime:
        print(f"Could not get recommendations for '{anime_title}'. Skipping evaluation.")
        continue

    # Determine "relevant" anime for the current test case
    # Get the index of the current anime
    if anime_title not in df_anime['name'].values:
         print(f"Anime '{anime_title}' not found in dataframe for relevance calculation. Skipping.")
         continue

    idx = df_anime[df_anime['name'] == anime_title].index[0]

    # Find anime with a similarity score above a threshold (e.g., 0.7) as relevant
    # Exclude the anime itself
    relevant_indices = [i for i, score in enumerate(cosine_sim[idx]) if score > 0.7 and i != idx]
    relevant_anime = df_anime['name'].iloc[relevant_indices].tolist()
    #print(f"Relevant: {relevant_anime}") # Uncomment to see relevant anime

    if not relevant_anime:
        print(f"No relevant anime found for '{anime_title}' based on the similarity threshold. Skipping evaluation for this item.")
        continue

    # Calculate metrics for the current test case
    precision = calculate_precision(recommended_anime, relevant_anime)
    recall = calculate_recall(recommended_anime, relevant_anime)
    f1 = calculate_f1_score(precision, recall)

    print(f"  Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1_scores.append(f1)

# Step 5: Calculate and print the average metrics
if all_precisions:
    avg_precision = sum(all_precisions) / len(all_precisions)
    avg_recall = sum(all_recalls) / len(all_recalls)
    avg_f1_score = sum(all_f1_scores) / len(all_f1_scores)

    print("\nOverall Average Performance:")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-score: {avg_f1_score:.4f}")
else:
    print("\nNo successful evaluations were performed.")

Evaluating recommendations for: Kimi no Na wa.
  Precision: 1.0000, Recall: 0.0008, F1-score: 0.0017
Evaluating recommendations for: Fullmetal Alchemist: Brotherhood
  Precision: 1.0000, Recall: 0.0008, F1-score: 0.0017
Evaluating recommendations for: Steins;Gate
  Precision: 1.0000, Recall: 0.0008, F1-score: 0.0017
Evaluating recommendations for: Spirited Away
Anime with title 'Spirited Away' not found in the dataset.
Could not get recommendations for 'Spirited Away'. Skipping evaluation.
Evaluating recommendations for: Attack on Titan
Anime with title 'Attack on Titan' not found in the dataset.
Could not get recommendations for 'Attack on Titan'. Skipping evaluation.

Overall Average Performance:
Average Precision: 1.0000
Average Recall: 0.0008
Average F1-score: 0.0017


In [9]:
# 1. Explain the core concept of collaborative filtering
print("1. Core Concept of Collaborative Filtering:")
print("Collaborative filtering is a technique used by recommendation systems to predict user preferences or recommend items.")
print("It works by collecting preferences or taste information from many users (collaborating) to predict what a particular user might like.")
print("The underlying assumption is that if two users liked the same items in the past, they are likely to like other items in the future.")
print("Alternatively, if two items are liked by the same users, they are likely similar and one could be recommended if the user liked the other.")
print("\n")

# 2. Describe user-based collaborative filtering
print("2. User-Based Collaborative Filtering:")
print("User-based collaborative filtering, also known as user-to-user or memory-based collaborative filtering, makes recommendations by finding users who are similar to the active user.")
print("Similarity between users is typically calculated based on their past interactions or ratings of items.")
print("Once a set of similar users is identified, the system recommends items that these similar users liked or rated highly, but which the active user has not yet interacted with.")
print("Steps involved:")
print("  a. Find users similar to the active user.")
print("  b. Identify items that similar users liked.")
print("  c. Recommend those items to the active user, excluding items the user has already seen.")
print("\n")

# 3. Describe item-based collaborative filtering
print("3. Item-Based Collaborative Filtering:")
print("Item-based collaborative filtering, also known as item-to-item or model-based collaborative filtering, makes recommendations by finding items that are similar to the items the active user has already liked or interacted with.")
print("Similarity between items is calculated based on how users have rated or interacted with those items.")
print("If a user liked Item A, the system looks for Item B that is similar to Item A (because many users who liked Item A also liked Item B) and recommends Item B.")
print("Steps involved:")
print("  a. Build a model of item similarity based on user interactions.")
print("  b. Identify items the active user has liked.")
print("  c. Find items similar to the user's liked items.")
print("  d. Recommend those similar items to the active user, excluding items the user has already seen.")
print("\n")

# 4. Highlight the key differences and potential trade-offs
print("4. Key Differences and Trade-offs:")
print("  - Approach: User-based focuses on finding similar users, while Item-based focuses on finding similar items.")
print("  - Calculation: User-based similarity is based on user profiles/ratings across items. Item-based similarity is based on item profiles/ratings across users.")
print("  - Scalability: User-based can suffer from scalability issues as the number of users grows (the 'cold start' problem for new users). Item-based tends to be more stable as item similarity changes less frequently than user tastes.")
print("  - Sparsity: User-based can be less effective with sparse data (few ratings per user), while Item-based can handle sparsity better by focusing on item relationships.")
print("  - Explainability: Item-based recommendations can sometimes be easier to explain to users (e.g., 'because you liked X, you might like Y').")
print("  - Maintenance: Item-based models often require less frequent updates than user-based models.")

1. Core Concept of Collaborative Filtering:
Collaborative filtering is a technique used by recommendation systems to predict user preferences or recommend items.
It works by collecting preferences or taste information from many users (collaborating) to predict what a particular user might like.
The underlying assumption is that if two users liked the same items in the past, they are likely to like other items in the future.
Alternatively, if two items are liked by the same users, they are likely similar and one could be recommended if the user liked the other.


2. User-Based Collaborative Filtering:
User-based collaborative filtering, also known as user-to-user or memory-based collaborative filtering, makes recommendations by finding users who are similar to the active user.
Similarity between users is typically calculated based on their past interactions or ratings of items.
Once a set of similar users is identified, the system recommends items that these similar users liked or rated